In [1]:
import time

### Load Data

In [2]:
# Parse the source
with open('THUOCL_chengyu.txt', 'r', encoding='utf-8') as source:
    idioms = [i for i in map(lambda x: x.split()[0], source.readlines()) if len(i) <= 4]

nodes = set([i[0] for i in idioms] + [i[-1] for i in idioms])
edges = [{"src": i[0], "dst": i[-1], "idiom": i} for i in idioms]

#### Create Graph

In [3]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", password="12345678")

cql = "CREATE \n" + ",".join("(%s : Node {char: \"%s\"})\n"%(n, n) for n in nodes)
for e in edges:
    cql += ",(%s)-[:idiom {idiom: '%s'}]->(%s)\n"%(e['src'], e["idiom"], e['dst'])

start = time.time()
graph.run(cql)
print("created graph, time elapsed: %.2fs"%(time.time() - start))

created graph, time elapsed: 9.14s


___
### Query Graph
##### Find next word

In [4]:
next_word = lambda word: graph.run("match ()-[:idiom {idiom: '%s'}]->()-[e:idiom]->() return e.idiom"%word)

df_next = next_word("坚定不移").to_data_frame()

print("found %i idioms"%len(df_next))
df_next.head()

found 3 idioms


,e.idiom
0,移山填海
1,移花接木
2,移风易俗


##### Find path

In [5]:
def find_path(idiom, length, limit):
    cql = "match ()-[e:idiom {idiom: '%s'}]->()"%idiom
    path = "".join(["-[e%i:idiom]->()"%i for i in range(length)])
    ret = ", ".join(["e%i.idiom"%i for i in range(length)])
    return graph.run(cql + path + " return e.idiom, " + ret + " limit %i"%limit).to_data_frame()

In [6]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=5, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 6833 paths, time elapsed: 0.22s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom
0,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐
1,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子虚乌有
2,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子丑寅卯
3,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子子孙孙
4,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子曰诗云
5,坚定不移,移山填海,海不扬波,波谲云诡,诡谲多变,变本加厉
6,坚定不移,移山填海,海不扬波,波谲云诡,诡谲多变,变幻无常
7,坚定不移,移山填海,海不扬波,波谲云诡,诡谲多变,变动不居
8,坚定不移,移山填海,海不扬波,波谲云诡,诡谲多变,变化莫测
9,坚定不移,移山填海,海不扬波,波谲云诡,诡谲多变,变化多端


In [7]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=10, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 100000 paths, time elapsed: 8.89s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom,e5.idiom,e6.idiom,e7.idiom,e8.idiom,e9.idiom
0,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成家立业,业精于勤,勤学苦练
1,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成家立业,业精于勤,勤学好问
2,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成家立业,业精于勤,勤勤恳恳
3,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成家立业,业精于勤,勤能补拙
4,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成家立业,业精于勤,勤工俭学
5,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成败萧何,何患无辞,辞旧迎新
6,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成败萧何,何必当初,初露头角
7,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成败萧何,何必当初,初生之犊
8,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成败萧何,何必当初,初试锋芒
9,坚定不移,移山填海,海不扬波,波平浪静,静若处子,子为父隐,隐隐约约,约定俗成,成败萧何,何必当初,初露锋芒
